In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
repo_dir = "/Users/ameliachu/repos/nlu-reddit-toxicity-dataset"
results_dir = f"{repo_dir}/artifacts/results"
date = '2022-05-08'
master_data_location  = f"{repo_dir}/data/labelled_master_data_{date}.csv"

In [3]:
labels = ['severe_toxicity', 'toxicity', 'identity_attack', 'insult', 'profanity', 'threat']

truth_labels = {label : f'y_truth_{label}' for label in labels}
pred_labels = {label : f'y_pred_{label}' for label in labels}

selected_columns = ['example_id'] + labels

In [4]:
human_2_pd = pd.read_csv(f"{results_dir}/baseline_human-2_2022-05-07.csv") 

In [5]:
test_set_ids = list(human_2_pd['example_id'].values)

In [6]:
truth_set = pd.read_csv(master_data_location)
truth_set = truth_set[truth_set['example_id'].isin(test_set_ids)].dropna().drop_duplicates(subset='example_id',keep='first')
truth_set = truth_set[selected_columns]
truth_set_renamed = truth_set.rename(columns=truth_labels)

In [7]:
len(truth_set)

100

In [9]:
human_2_preds = human_2_pd[selected_columns]
human_2_preds_renamed = human_2_preds.rename(columns=pred_labels)

In [10]:
overall_human_2_preds = pd.melt(human_2_preds, id_vars = ['example_id'], value_vars=labels).rename(columns={'value':'y_pred'})
overall_truth_set = pd.melt(truth_set, id_vars = ['example_id'], value_vars=labels).rename(columns={'value':'y_truth'})


In [12]:
y_labels = truth_set_renamed.merge(human_2_preds_renamed, on='example_id')
overall_y_labels = overall_truth_set.merge(overall_human_2_preds, on='example_id')

In [14]:
results_list =  []
label = 'profanity'
for label in labels:
    y_truth_label = f'y_truth_{label}'
    y_pred_label = f'y_pred_{label}'
    precision, recall, f1_score, support = precision_recall_fscore_support(y_labels[y_truth_label].values, y_labels[y_pred_label].values)

    eval_results = {
        'attribute': label,
        'precision': precision[1],
        'recall': recall[1],
        'f1_score': f1_score[1]
    }

    results_list.append(eval_results)

/Users/ameliachu/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
precision, recall, f1_score, support = precision_recall_fscore_support(overall_y_labels['y_truth'], overall_y_labels['y_pred'])
eval_results = {
        'attribute': 'overall',
        'precision': precision[1],
        'recall': recall[1],
        'f1_score': f1_score[1]
    }
results_list.append(eval_results)

In [16]:
pd.DataFrame(results_list)

,attribute,precision,recall,f1_score
0,severe_toxicity,0.000000,0.000000,0.000000
1,toxicity,1.000000,0.200000,0.333333
2,identity_attack,0.875000,0.350000,0.500000
3,insult,0.696970,0.605263,0.647887
4,profanity,0.956522,0.800000,0.871287
5,threat,0.000000,0.000000,0.000000
6,overall,0.442105,0.269231,0.334661


In [ ]:
y_labels.isna()

In [ ]:
y_labels.to_csv(f"{results_dir}/tmp.csv")